In [62]:
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import logging
from tqdm import tqdm
import time
import datetime

In [ ]:
title = "Software Developer"  
location = "Linköping"
NUMBER_OF_JOBS_TO_FETCH = 600
SAVE_DATA = True

In [64]:
import os
os.makedirs('./results', exist_ok=True) # will dump results here

In [65]:
from collections import defaultdict
prg_skills = pd.read_csv('./skills.csv')['skills'].to_list()
prg_skills = set(prg_skills)

users_of_languages = defaultdict(int)
for l in prg_skills:
    users_of_languages[l] = 0

In [66]:
job_list = []

In [67]:
for start in tqdm(range(0, NUMBER_OF_JOBS_TO_FETCH, 25)):
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&distance=25&f_TPR=&f_WT=1&start={start}"

    response = requests.get(list_url)

    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")
    page_jobs = list_soup.find_all("li")
        
    id_list = []

    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        job_id = base_card_div.get("data-entity-urn").split(":")[3]
        id_list.append(job_id)
    
    for job_id in id_list:
        job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        
        job_response = requests.get(job_url)
        if job_response.status_code != 200:
            logging.warning(f"Failed to retrieve job posting {job_id}: Status code {job_response.status_code}")
            continue
        job_soup = BeautifulSoup(job_response.text, "html.parser")
        
        job_post = {}
        
        try:
            job_post["job_title"] = job_soup.find("h2", {"class":"top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
        except:
            job_post["job_title"] = None
            
        try:
            job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
        except:
            job_post["company_name"] = None

        try: 
            job_post["location"] = job_soup.find("span", {"class": "topcard__flavor topcard__flavor--bullet"}).text.strip().split(",")[0].strip()
        except:
            job_post["location"] = None
            
        try:
            job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            job_post["time_posted"] = None
            
        try:
            job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip().replace(" applicants", "")
        except:
            job_post["num_applicants"] = 0 

        for lang in prg_skills:
            if f"{lang.lower()}" in job_soup.find("div", {"class": "show-more-less-html__markup show-more-less-html__markup--clamp-after-5 relative overflow-hidden"}).text.lower():  
                users_of_languages[lang] += 1 
        job_list.append(job_post)
        time.sleep(random.uniform(1, 3)) # Sleep between requests to avoid rate limiting 

100%|██████████| 24/24 [01:47<00:00,  4.49s/it]


## Job postings 

In [68]:
jobs_df = pd.DataFrame(job_list)
print(f"Total job postings collected: {len(jobs_df)}")
jobs_df.head(15)

Total job postings collected: 40


,job_title,company_name,location,time_posted,num_applicants
0,Algoritmutveckling / Reglerteknik / Modellbase...,Combine,Linköping,2 weeks ago,0
1,System developer to Linköping!,TMC Sweden,Linköping,6 days ago,26
2,Testare inom mjukvara,Professional Galaxy AB,Linköping,8 months ago,39
3,Utvecklare Java,Nexer Group,Linköping,2 days ago,0
4,"Software Developer, Backend - Sectra Medical",Sectra,Linköping,2 weeks ago,192
5,Software Engineer,Voyado,Norrköping,6 days ago,29
6,Software Developer,Väderstad AB,Linköping,2 weeks ago,32
7,Java Software Engineer,Piktiv AB,Linköping,1 week ago,34
8,Erfaren .Net-utvecklare,XLENT,Norrköping,1 month ago,0
9,AI Software Developer,Knowit,Linköping,3 days ago,0


In [69]:
if SAVE_DATA:
    jobs_df.to_csv(f'./results/jobs_{title}_{location}_{datetime.datetime.now().strftime("%Y-%m-%d")}.csv', index=False)

## Skills usage in job postings 

In [73]:
skill_usage = pd.DataFrame.from_dict(users_of_languages, orient='index', columns=['Number of Job Postings']).reset_index().rename(columns={'index': 'Programming Skill'})
skill_usage.drop(skill_usage[skill_usage['Number of Job Postings'] == 0].index, inplace=True)
skill_usage = skill_usage.sort_values(by='Number of Job Postings', ascending=False)
skill_usage = skill_usage.reset_index(drop=True)
skill_usage.head(20)

,Programming Skill,Number of Job Postings
0,C++,12
1,Python,12
2,Git,12
3,C#,10
4,Azure,10
5,Kubernetes,8
6,SQL,8
7,Docker,7
8,CI/CD,7
9,Java,7


In [74]:
if SAVE_DATA:
    skill_usage.to_csv(f'./results/skills_{title}_{location}_{datetime.datetime.now().strftime("%Y-%m-%d")}.csv', index=False)